El codigo convierte un pdf de liquidaciones de empledos y un archivo de excel en varios archivos pdf, separados por cedulas su funcionamiento es convertir el pdf en imagenes cada hoja, para modificarlos segun convenga, separ las hojas por cedula y asi...
Necesitas instalar poppler y Teseract-ocr para que funcione, y además en la misma carpeta donde corres el codigo debes de guardar arial.ttf es la fuente con la que escribirá en el pdf.

In [1]:
from pdf2image import convert_from_path
import pytesseract
from PIL import ImageFont, ImageDraw, Image
import numpy as np
import pandas as pd
import os
import numpy as np
#librerias (instala todas por si acaso) pip install..

#Ruta de los archivos, solo se deben remplazar las rutas por las de los arcivos para que puedan ser leidos
#el programa ya identifica el pdf que le llega, y el formato del libro xslx es el mismo
#ruta out es donde quieres guardar los archivos, solo necesitas una carpeta vacia donde guardarlos
#solo ajusta las rutas y dejalo corer
Ruta_pdf = r"C:\Users\danie\OneDrive\Escritorio\Jupyter Data\Real test\Data liquidaciones\2023-12-26.pdf"
Ruta_libro = r"C:\Users\danie\OneDrive\Escritorio\Jupyter Data\Real test\Data liquidaciones\Liquidaciones 26-12-2023 Datos a Modificar en PDF FINAL.xlsx"
Rutaout = r"C:\Users\danie\OneDrive\Escritorio\Jupyter Data\Real test\Data liquidaciones out/" 

#no hay que entender mucho de aqui esta solo es una clase para utiizar los programas
class LiquidationPDF: 
    def __init__(self):
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract' 
        #ruta de ubicacion de tesseract-OCR. Este un programa para analizar texto de imagenes se necesita para la libreria pytesseract
        #ajusta la ruta a pyteseract debes colocar la ruta de tu tesseract-ocr probablemente lo deje cargado en una carpeta para que lo descargues
        self.images = [] #lista vacia donde se guardara cada hoja del pdf 

        #self.font parece inutil y lo es, pero dejalo sol para que no te de problemas 
        self.font = ImageFont.truetype("./arial.ttf", 27) #es el tipo de letra con la que se va a remplazar el archivo y el tamaño
    
    def read_pdf(self, path):#se utiliza para leer la ruta del pdf y habilitar poppler
        #poppler te lee el pdf y lo convierte a imagenes para tratarlas mejor
        for image in convert_from_path(path, poppler_path=r'C:\Program Files\poppler-23.08.0\Library\bin'): 
            #ajustar la ruta a poppler te dejare poppler junto a tesseract-ocr si puedes guardalo en program files
            self.images.append(np.array(image)) #pega imagen del pdf a self.images (la lista)
    
    #read id se encarga de leer el id de cada imagen para clasificar cada imagen por la cedula que posea
    def read_id(self, img): #en img recorta el array (la imagen convertida a matriz) y utiliza pytesseract para leer la cedula del pdf y retornarla
        return pytesseract.image_to_string(img[280:320, 1390:1600])[:-1]
    
    def replaceDate(self, img, date, number):
        #borra la informacion del pdf convertido a imagen coloreando los pixeles en blanco
        img[533:593, 1090:1300] = 255 #Fecha a modificar
        img[610:640, 440:900] = 255 #Dias laborados
        img_pil = Image.fromarray(img) #convierte el arrary en imagen
        draw = ImageDraw.Draw(img_pil) #dibuja sobre la imagen 
        draw.text((1126, 546),  date, font = self.font, fill = (0, 0, 0))# le rescribe los datos number y date en la imagen)
        draw.text((546,608),number+" días laborados.", font =  ImageFont.truetype("./arial.ttf", 26.5), fill = (0, 0, 0))
        return img_pil
    
    def getImages(self):#devuelve la lista 
        return self.images
            
    def read_idM(self, img): #es el mismo codigo de arriba pero ajustado para otro tipo de formato pdf 
        return pytesseract.image_to_string(img[380:420, 1350:1600])[:-1]
    
    def replaceDateM(self, img, date, number): #mismo codigo de arriba ajustado para otro formato
        img[630:660, 1090:1250] = 255 #date
        img[690:725, 520:900] = 255 # lab 
        img_pil = Image.fromarray(img)
        draw = ImageDraw.Draw(img_pil)
        draw.text((1100, 635),  date, font = ImageFont.truetype("./arial.ttf", 26.5), fill = (0, 0, 0))
        draw.text((533,693),number+" días laborados.", font =  ImageFont.truetype("./arial.ttf", 26.5), fill = (0, 0, 0))
        return img_pil
    

In [2]:
DC_CDL= pd.read_excel(Ruta_libro, usecols=[0,1,3]) #lee el archivo excel cedula, dias, fechas
DC_CDL[DC_CDL.columns[1]] = DC_CDL[DC_CDL.columns[1]].astype(str) #la convierte a str ya no recuerdo porque)
DC_CDL[DC_CDL.columns[2]] = DC_CDL[DC_CDL.columns[2]].apply(lambda x: '{:,}'.format(x)) #les camia el formato a los numeros 1000 -> 1,000
DC_CD = dict(zip(DC_CDL[DC_CDL.columns[0]], DC_CDL[DC_CDL.columns[1]])) #crea un diccionario que le asigna a cada cedula del excel dias
DC_DL = dict(zip(DC_CDL[DC_CDL.columns[0]], DC_CDL[DC_CDL.columns[2]])) #crea un diccionario que le asigna a cada cedula del excel fechas
processor = LiquidationPDF() #le asigna la clase
processor.read_pdf(Ruta_pdf) #lee el pdf y los convierte a imagenes
Getimages = processor.getImages() #guarda en get images las fotos 
img = Getimages[-1] #quita la ultima imagen por que es una pagina en blanco 
img[70:120, 630:1030].shape #indica las dimenciones del archivo, se puede quitar pero como no afecta en nada se deja por si acaso
mi_array = np.zeros((50, 400, 3), dtype=np.uint8)
mi_array.fill(255) #es un aray vacio o blanco 
#el siguiente codigo es un if que separa los formatos, me dice que tipo de pdf estoy trabajando
if np.array_equal(mi_array,img[70:120, 630:1030]): #se comparan para verificar el tipo de formato del pdf
    #algo asi como: mi formato es 'A' entonces en cierta posición del pdf sé que habran zonas que son totalmente blancas si se cumple eso trabajare los ajustes de formato A
    print('Entro en M')# para saber en que formato esta trabajando, se puede omitir
    ids = {} #diccionario donde guardare las cedulas con las imagenes key = cedula value = Imagen
    for image in Getimages: #for la lista con todas las imagenes
        temp_id = processor.read_idM(image) #deme el la cedula en la imagen 
        if temp_id in ids.keys(): #si la key ya pertenece agreguele la imagen si no, cree una key con la imagen 
            ids[temp_id].append(image)
        else:
            ids[temp_id] = []
            ids[temp_id].append(image)
    #asi obtengo con la key cedula todas las imagenes que le correspondan
    for ced in ids.keys():
        print(ced) #imprime la cedula como metodo de verificacion que se rompa el codigo
        real_lab = DC_DL.get(int(ced), None) #obtiene los valores de dias laburados y cedula que deben de ser devueltos
        real_ced = DC_CD.get(int(ced), None)
        print(real_lab, real_ced) #lo imprime para verificar que no agregará un Nan no deseado a alguna hoja
        if real_ced != None:
            print(True) #se usa para verficar que todo este bien 
            for index, image in enumerate(ids[ced]):
                ids[ced][index] = processor.replaceDate(image, real_ced, real_lab).convert('RGB') #remplaza la cedula y lab, ademas convierte la imagen en un formato que se puede exportar a pdf
            if len(ids[ced]) > 0:#si el pdf mide 1, solo lo guarda, si mide más concatena las imagenes y las vuelve un pdf
                ids[ced][0].save(Rutaout+ced+".pdf", save_all=True, append_images=ids[ced][1:])
            else:
                ids[ced][0].save(Rutaout+ced+".pdf", save_all=True)
        else:
            continue #esta en continue, por que ellos necesitan que el pdf solo mande los que se modifican, los que no se ignoran (lo dejo por si acaso)
            print(False) #si es false, no transforma nada pero aun asi pega las imagenes y guarda los pdf's
            for index, image in enumerate(ids[ced]):
                ids[ced][index] = Image.fromarray(image).convert('RGB')
            if len(ids[ced]) > 0:
                ids[ced][0].save(Rutaout+ced+".pdf", save_all=True, append_images=ids[ced][1:])
            else:
                ids[ced][0].save(Rutaout+ced+".pdf", save_all=True)
    print('termino:p')
else: #es el mismo codigo pero con otros parametros 
    print('NO Entro en M')
    ids = {}
    for image in Getimages:
        temp_id = processor.read_id(image)
        if temp_id in ids.keys():
            ids[temp_id].append(image)
        else:
            ids[temp_id] = []
            ids[temp_id].append(image)
    id2 = ids
    id2.pop('') #como es un pdf con otro formato toca eliminar basuritas extrs que genera este pdf, nada más, de resto es lo mismo
    id2.keys()#omitible pero lo dejo por que lo usé para testear que todo funcionase bien 
    ids = id2
    for ced in ids.keys():
        print(ced)
        real_lab = DC_DL.get(int(ced), None)
        real_ced = DC_CD.get(int(ced), None)
        print(real_lab, real_ced)
        if real_ced != None:
            print(True)
            for index, image in enumerate(ids[ced]):
                ids[ced][index] = processor.replaceDate(image, real_ced, real_lab).convert('RGB')
            if len(ids[ced]) > 0:
                ids[ced][0].save(Rutaout+ced+".pdf", save_all=True, append_images=ids[ced][1:])
            else:
                ids[ced][0].save(Rutaout+ced+".pdf", save_all=True)
        else:
            continue
            print(False)
            for index, image in enumerate(ids[ced]):
                ids[ced][index] = Image.fromarray(image).convert('RGB')
            if len(ids[ced]) > 0:
                ids[ced][0].save(Rutaout+ced+".pdf", save_all=True, append_images=ids[ced][1:])
            else:
                ids[ced][0].save(Rutaout+ced+".pdf", save_all=True)
    print('termino:p')#es para saber que no se ha colgado funcionando
    #no revisas el false por que simplemente no los agrega, y avisa siempre cuantos pdf's imprimió para saber que no hubo ningun error

    #ERRORES 
    #EXCEL Que el formato de excel sea enviado de forma diferente, todas las celdas deben de mantener un formato especifcico por que asi lo lee python
    #no cambien celdas numero por celdas texto, ni cambien el orden de las columnas, solo agreguen bajo el mismo formato, lo que se desee cambiar
    #nada más

    #PDF que el archivo pdf, sea un formato nuevo, en ese caso debes de copiar nuevamente el codigo de Image.fromarray get ID para que se ajuste al nuevo formato
    #algo asi como crear un Image.fromarray2() para que el codigo no se te muera y crear un if nuevo que verifique la nueva condicion y utilice esos parametros cuando sea necesario
    
    #Teseract-ocr / Poppler ----> instalar teseract o poppler es una mamera, porque para que no te pida la ruta, te pide instalarse como variable del sistema
    #lo cual no funciona, es mas facil pegar esta carpeta en archivos del sistema, y darle la ruta seleccionada pero si no funciona 
    #dejo un tutorial por si acaso https://ucd-dnp.github.io/ConTexto/versiones/master/instalacion/instalacion_popple_teseract.html
    


NO Entro en M
1019145048
None None
1121934186
310 2023-12-19
True
39642295
9,519 2023-12-15
True
1035391600
2,453 2023-12-14
True
1061113790
430 2023-12-17
True
1007982611
113 2023-12-10
True
1016004006
4,258 2023-12-18
True
94062677
4,152 2023-12-12
True
1000506565
701 2023-12-18
True
1143858314
1,171 2023-12-12
True
52862856
40 2023-12-12
True
1022416170
283 2023-12-15
True
1000089273
48 2023-12-20
True
8407585
5,378 2023-12-15
True
1013688039
21 2023-11-30
True
1233191109
300 2023-12-16
True
1078370178
2,040 2023-12-13
True
1007727281
9 2023-12-15
True
1005753260
None None
1007395386
None None
1010030163
None None
1075210680
3,561 2023-12-12
True
1075674736
727 2023-12-12
True
52343930
4,071 2023-12-13
True
1000189981
273 2023-12-12
True
1001782353
53 2023-12-16
True
73162000
1,526 2023-11-28
True
1053785573
620 2023-12-20
True
1031610181
124 2023-12-07
True
1012432928
488 2023-12-13
True
1012383439
630 2023-12-18
True
52907709
5,549 2023-12-13
True
1012456412
636 2023-12-10
True
10